In [1]:
"""
This frankenstein code borrows heavily from the work of: 
https://developers.google.com/kml/articles/geotagsimple
* Copyright 2008 Google Inc. All Rights Reserved.

Reads the EXIF headers from geo-tagged photos and creates a KML file with
a PhotoOverlay element for each file. Requires the open source EXIF.py file
downloadable at:
http://sourceforge.net/projects/exif-py/
  __author__ = 'mmarks@google.com (Mano Marks)'
  
Purpose: 
- Extract GPS data from exif headers of geotagged photo
- Writes a .kml file using this GPS data and the geotagged photo name. 

Future work:
- Expand to multiple folders. Presently only for a single folder. Writes a .kml for each subdirectory.
- Filter for only photo filetypes.
- Writes .csv or text files with filename to extract gps data from file. 
"""

"\nThis frankenstein code borrows heavily from the work of: \nhttps://developers.google.com/kml/articles/geotagsimple\n* Copyright 2008 Google Inc. All Rights Reserved.\n\nReads the EXIF headers from geo-tagged photos and creates a KML file with\na PhotoOverlay element for each file. Requires the open source EXIF.py file\ndownloadable at:\nhttp://sourceforge.net/projects/exif-py/\n  __author__ = 'mmarks@google.com (Mano Marks)'\n  \nPurpose: \n- Extract GPS data from exif headers of geotagged photo\n- Writes a .kml file using this GPS data and the geotagged photo name. \n\nFuture work:\n- Expand to multiple folders. Presently only for a single folder. Writes a .kml for each subdirectory.\n- Filter for only photo filetypes.\n- Writes .csv or text files with filename to extract gps data from file. \n"

In [1]:
import exifread as EXIF
import os
import sys
from pykml.factory import KML_ElementMaker as KML
from lxml import etree

In [11]:
def GetFile(file_name):
    """Handles opening the file.
    Args:
    file_name: the name of the file to get
    Returns:
    A file
    """

    the_file = None
    try:
        the_file = open(file_name, 'rb')
    except IOError:
        the_file = None
    return the_file



def GetHeaders(the_file):
    """Handles getting the EXIF headers and returns them as a dict.
    Args:
    the_file: A file object
    Returns:
    a dict mapping keys corresponding to the EXIF headers of a file.
    """
    data = EXIF.process_file(the_file, details=False)  # , 'UNDEF', False, False, False
    return data



def DmsToDecimal(degree_num, degree_den, minute_num, minute_den,
                 second_num, second_den):
    """Converts the Degree/Minute/Second formatted GPS data to decimal degrees.
    Args:
    degree_num: The numerator of the degree object.
    degree_den: The denominator of the degree object.
    minute_num: The numerator of the minute object.
    minute_den: The denominator of the minute object.
    second_num: The numerator of the second object.
    second_den: The denominator of the second object.
    Returns:
    A deciminal degree.
    """
    degree = float(degree_num)/float(degree_den)
    minute = float(minute_num)/float(minute_den)/60
    second = float(second_num)/float(second_den)/3600
    return degree + minute + second

def GetGps(data):
    """Parses out the GPS coordinates from the file.
    Args:
    data: A dict object representing the EXIF headers of the photo.
    Returns:
    A tuple representing the latitude, longitude, and altitude of the photo.
    """
    lat_dms = data['GPS GPSLatitude'].values
    long_dms = data['GPS GPSLongitude'].values
    latitude = DmsToDecimal(lat_dms[0].num, lat_dms[0].den,
                            lat_dms[1].num, lat_dms[1].den,
                            lat_dms[2].num, lat_dms[2].den)

    longitude = DmsToDecimal(long_dms[0].num, long_dms[0].den,
                            long_dms[1].num, long_dms[1].den,
                            long_dms[2].num, long_dms[2].den)
    # Error here handled earlier, in main loop
    if data['GPS GPSLatitudeRef'].printable == 'S': latitude *= -1
    if data['GPS GPSLongitudeRef'].printable == 'W': longitude *= -1
    altitude = None
    try:
        alt = data['GPS GPSAltitude'].values[0]
        altitude = alt.num/alt.den
        if data['GPS GPSAltitudeRef'] == 1: altitude *= -1
    except KeyError:
        altitude = 0
    return latitude, longitude, altitude

In [15]:
##Make KML with pykml using https://pythonhosted.org/pykml/examples/kml_reference_examples.html
def CreateKmlDoc(kmlname, usr_desc):
    kml_kml = KML.kml()

    kml_doc = KML.Document(
        KML.name(kmlname),
        KML.description(usr_desc)
        )
    return kml_kml, kml_doc
    
def CreatePhotoOverlay(kml_doc, file_name, the_file, file_iterator):
    photo_id = 'photo{}'.format(file_iterator)
    data = GetHeaders(the_file)
    coords = GetGps(data)
    
    # Determines the proportions of the image and uses them to set FOV.
    width = float(data['EXIF ExifImageWidth'].printable)
    length = float(data['EXIF ExifImageLength'].printable)
    lf = str(width/length * -20.0)
    rf = str(width/length * 20.0)
    
    PO = KML.PhotoOverlay(
        KML.name(file_name),
        KML.id(photo_id),
        KML.Camera(
            KML.longitude(coords[1]),
            KML.latitude(coords[0]),
            KML.altitude(10),
            KML.tilt(90)
            ),
        KML.styleUrl('camera'),
        KML.Icon(
            KML.href(file_name)    #usr_path + '\\'+ file_name
        ),
        KML.ViewVolume(
            KML.near(50),
            KML.leftFov(lf),
            KML.rightFov(rf),
            KML.bottomFov(-20),
            KML.topFov(20)
        ),
        KML.Point(
            KML.coordinates('{},{},{}'.format(coords[1], coords[0], coords[2]))
        )
    )
    
    kml_doc.append(PO)

In [25]:
# Read files in directory
# Input necessary (usrcode == 1), input hardcoded (Adv Geomorph) (2)
usrcode = 2

if usrcode ==1: 
    usr_path = raw_input("Enter directory path:")
    the_file = str(usr_path)
elif usrcode == 2:
    usr_path = 'C:\Users\\nvmille1\OneDrive\Research\Landuse\Field_2017\Mescal Mts-06-14-17'
    usr_desc = 'Mescal Mts Field 2017-6-14-17'
else: 
    usr_path = 'C:\Users\\nariv\OneDrive\Research\Landuse\Code\Photos_to_KML\AdvGeomorphFieldTrip'
    usr_desc = u'Adv Geomorph 2017 visits Seneca Falls, \n \
                the Salt River, \n \
                Rim gravels preserved beneath basalt flows, exposed in roadcut \n \
                Whiteriver, more basalts (some on hilltops, some in the valley) preserving Rim Gravels \n \
                Pinetop (dinner), \n \
                the 260 Overlook from the Rim -- Main question: Is the topographic relief due to erosion, or tectonics? \n \
                Bear Flat Road and Tonto Creek -- investigating the knickpoint in the Tonto Creek, and the low-relief terrain transitioning to a canyon, \n \
                more gravels (sandier) between Pine and Payson - younger gravels with dominant flow direction to the East, topped by basalt, \n \
                overlook of the Rye Valley - multiple stages of filling and emptying the basin-and-range topography \
                in the relatively tectonically quiet times the past ~5Ma,\n \
                and a roadcut of an andesite flow (~23 Ma) topped by cobbles and valley fill.'        

photo_counter = 0
filenames = []


In [27]:


for root, dirs, files in os.walk(usr_path):
    kmlname = root.split('\\')[-1]
    fullkmlname = '{}\{}.kml'.format(root,kmlname)
    
    file_iterator = 1
    kml_kml, kml_doc = CreateKmlDoc(fullkmlname, kmlname)
    
    for f in files:
        photo_counter += 1
        the_file = GetFile(usr_path + '\\' + f)
        if the_file is None:
            print "'{}' is unreadable\n".format(f)
            continue

        try:
            CreatePhotoOverlay(kml_doc, f, the_file, file_iterator)
        except KeyError:
            print"'{}' has no GPS GPSLatitudeRef".format(f)
            continue
        file_iterator += 1
        the_file.close()
    
#After all photo overlays have been written, append the doc to the outermost layer, kml_kml
kml_kml.append(kml_doc)
  
fld_contents = etree.tostring(kml_kml, pretty_print=True)    
kml_file = open(fullkmlname, 'w')
kml_file.write(fld_contents)
kml_file.close()


print 'Finished writing {} photo overlays to {}.'.format(file_iterator, kmlname)

[33, 9, 1791/50]
[33, 9, 1791/50]
[33, 9, 1791/50]
[33, 9, 153/4]
[33, 9, 153/4]
[33, 9, 153/4]
[33, 9, 1359/25]
[33, 10, 282/25]
[33, 10, 282/25]
[33, 10, 641/50]
[33, 10, 1103/100]
[33, 10, 1103/100]
[33, 10, 1103/100]
[33, 10, 1103/100]
[33, 10, 1103/100]
[33, 10, 304/25]
[33, 10, 607/50]
[33, 10, 1473/100]
[33, 10, 1473/100]
[33, 10, 1473/100]
[33, 10, 1819/100]
[33, 10, 469/25]
[33, 10, 469/25]
[33, 10, 961/50]
[33, 10, 961/50]
[33, 10, 961/50]
[33, 10, 193/10]
[33, 10, 1929/100]
[33, 10, 1929/100]
[33, 10, 1929/100]
[33, 10, 967/50]
[33, 10, 967/50]
[33, 10, 967/50]
[33, 10, 1839/100]
[33, 10, 1843/100]
[33, 10, 1849/100]
[33, 10, 37/2]
[33, 10, 37/2]
[33, 10, 923/50]
[33, 10, 331/20]
[33, 10, 331/20]
[33, 10, 331/20]
[33, 10, 1651/100]
[33, 10, 1651/100]
[33, 10, 1481/100]
[33, 10, 1481/100]
[33, 10, 1489/100]
[33, 10, 372/25]
[33, 10, 372/25]
[33, 10, 149/10]
[33, 10, 149/10]
[33, 10, 241/20]
[33, 10, 241/20]
[33, 10, 241/20]
[33, 10, 241/20]
[33, 10, 1203/100]
[33, 10, 302/25]